In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import data and display 
df = pd.read_csv('time-series/zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [3]:
# more information on the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


We can see that there are more than 14,000 rows and 272 columns.  From printing the top couple of rows above, we can see that our data is presented in wide format.

In [4]:
# look at dataset, prior to dates to get a sense of what else is provided
df.iloc[:, :7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RegionID    14723 non-null  int64 
 1   RegionName  14723 non-null  int64 
 2   City        14723 non-null  object
 3   State       14723 non-null  object
 4   Metro       13680 non-null  object
 5   CountyName  14723 non-null  object
 6   SizeRank    14723 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 805.3+ KB


In [5]:
import folium
import json

In [13]:
# link to geojson data for us zipcodes
url = 'https://github.com/jgoodall/us-maps/blob/master/geojson/'
filename = 'zcta5.geo.json'
zip_geo = f'{url}{filename}'

In [ ]:
# test choropleth with one state


In [7]:
# create temporary dataframe, containing just our zipcode and size rank - - for now we will
# visualize based odn sizerank
temp_df = df[['RegionName', 'SizeRank']]
temp_df.head()

,RegionName,SizeRank
0,60657,1
1,75070,2
2,77494,3
3,60614,4
4,79936,5


In [9]:
# only keep the zipcodes which exist in our dataframe (temp_df)
geojson_keepers = [] # empty list to store geojson matches
num_zips_geojson = len(data['features']) # number of zipcodes in our geojson data
for i in range(num_zips_geojson):
    geojson_zip = int(data['features'][i]['properties']['ZCTA5CE10'])
    if geojson_zip in list(temp_df['RegionName']):
        geojson_keepers.append(data['features'][i])

In [16]:
# create new json file from cross-referenced zipcodes
new_json = dict.fromkeys(['type', 'features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geojson_keepers

In [17]:
# plot map
samp_map = folium.Map(location = [35, -90], zoom_start=4)
#samp_map

# 2. Data Preprocessing

In [5]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [6]:
# convert our dataframe
get_datetimes(df)

ValueError: time data RegionName doesn't match format specified